<a href="https://colab.research.google.com/github/Danieljuarez-cmd/Investigaci-n-de-Operaciones-/blob/main/EOQ_con_restricci%C3%B3n_de_almac%C3%A9n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EOQ Modelo de Cantidad Económica de Pedido con restricción de almacén
Alumno: Juárez Parangueo Edgar Daniel

Modelo de Cantidad Económica de Pedido con varios artículos y capacidad en el almacén limitada.

Tomamos como referencia para elaborar este código el ejemplo 11.2-3, sección 11.2.3, del libro Taha.

In [14]:
#Importamos las librerías necesarias
import sympy as sp
import numpy as np
import scipy.optimize as opt

Definimos las variables para el ejercicio, y las derivamos son Sympy.
Donde (k) es el costo de ordenar,(D) es la demanda,(h) es el costo de mantener el inventario, (y) es la cantidad a pedir , y (a) es el área ocupada por cada unidad de producto.

#Paso 1: Resolvemos sin restricción el problema del almacén.


In [15]:
# Variables
K, D, h, y = sp.symbols('K D h y', positive=True)
# Costo individual
C_ind = K*D/y + h*y/2
# Derivada
dCdy = sp.diff(C_ind, y)

# Solución óptima
y_star = sp.solve(dCdy, y)[0]
y_star


sqrt(2)*sqrt(D)*sqrt(K)/sqrt(h)

La solución analítica:

y* = sqrt(2 K D / h)

El resultado anterior es el la cantidad ecnómica sin restricción.


In [16]:
#Tomamos los datos del ejercicio del Taha

# Cantidad de artículos
n = 3
# Datos
K = np.array([10, 5, 15], dtype=float)
D = np.array([2, 4, 4], dtype=float)
h = np.array([0.30, 0.10, 0.20], dtype=float)
a = np.array([1, 1, 1], dtype=float)

# Capacidad del almacén
A = 25

Ahora obtenemos la cantidad económica de pedido de cada artículo por separado

In [17]:
y_eoq = np.sqrt(2*K*D/h)
y_eoq


array([11.54700538, 20.        , 24.49489743])

Verificamos con la restricción del probelma, donde veremos que si la suma exede A=25 l solución sin considerar la restricción no será factible

#Paso 2: Restricción del almacén


In [18]:
np.sum(a * y_eoq)


np.float64(56.0419028116243)

Notamos que la suma exede por mas del doble la cantidad factible entoces la cantidad sin restriccion no es factible.

#Paso 3: Resolvemos entonces con la restricción

In [19]:
#Definimos la función objetivo
def costo_total(y):
    total = 0.0
    for i in range(n):
        total += K[i]*D[i]/y[i] + h[i]*y[i]/2
    return total
#Restricción del almacén
cons = ({
    'type': 'eq',
    'fun': lambda y: np.sum(a*y) - A
})
#Definimos el punto inicial de la cantidad ecnoómica de pedido sin restricción
y0 = y_eoq.copy()
y0


array([11.54700538, 20.        , 24.49489743])

Los resultados anteriores son las cantidades óptimas a pedir pero exeden la cantidad óptima del almacén, entonces, optimizamos de tal forma que se minimiza el costo total con la restricción de almacén, consideramos que esta solución es equivalente al método de λ (lagrange) que viene en el ejemplo del Taha.

In [20]:
sol = opt.minimize(costo_total, y0, constraints=cons)
sol


     message: Optimization terminated successfully
     success: True
      status: 0
         fun: 13.623773560332124
           x: [ 6.340e+00  7.088e+00  1.157e+01]
         nit: 10
         jac: [-3.476e-01 -3.481e-01 -3.480e-01]
        nfev: 40
        njev: 10
 multipliers: [-3.480e-01]

In [21]:
#Obtenemos los resultados óptimos para validar los resultados del ejemplo del libro
y_opt = sol.x
costo_opt = sol.fun

print("Cantidades óptimas:")
for i in range(n):
    print("y_", i+1, "=", round(y_opt[i], 4))

print("\nCosto total mínimo:", round(costo_opt, 4))
print("Uso de almacén:", round(np.sum(a*y_opt), 4))


Cantidades óptimas:
y_ 1 = 6.3397
y_ 2 = 7.0878
y_ 3 = 11.5725

Costo total mínimo: 13.6238
Uso de almacén: 25.0


Resultados que están en el ejemplo del taha:

y1 ≈ 6.34  
y2 ≈ 7.09  
y3 ≈ 11.57  

Hacemos la comparación con los valores obtenidos con la librería scipy y vemos que coinciden con variaciones en los decimales.

#Conclusión
En resumen, se realizó un código con librerías para hacer mas eficiente y preciso la implementación del método de 3 pasos de Cantidad económica de pedido con restricción de almacén, donde primero calculamos la EOQ sin la restricción de almacén, hacemos la comparación con la capacidad de almacén, posteriormente resolvemos el problema considerando la restricción, en este caso usamos la optimización numérica con la función(minimize) de la librería Scipý, este método es equivalente a el método de multiplicadores de Lagrange.